In [41]:
# # # !pip install albumentations
# # !pip install --upgrade tensorflow keras

# !pip install --upgrade tensorflow tensorflow-addons keras


In [42]:

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
import albumentations as A
from albumentations.core.composition import OneOf
# from tensorflow_addons.losses import SigmoidFocalCrossEntropy

# Configuración de hiperparámetros
EPOCHS = 50
IMAGE_SIZE = (128, 128)
INPUT_SHAPE = (128, 128, 3)
SEED = 123
BATCH_SIZE = 64
BUFFER_SIZE = 350
FINE_TUNE_POINT = 100
LEARNING_RATE = 0.001

# Directorio del dataset
DATASET_DIR = '../arcgis-survey-images-new'


In [43]:

# Función para aplicar aumentación avanzada con Albumentations
def advanced_augmentation(image):
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.RandomGamma(gamma_limit=(80, 120), p=0.5),
        ], p=1),
        A.GaussNoise(p=0.2),
        A.MotionBlur(blur_limit=3, p=0.2),
        A.CLAHE(clip_limit=2, p=0.2),
    ])
    augmented = transform(image=image)
    return augmented["image"]


In [44]:
import tensorflow as tf

def mixup(data):
    x, y = data
    # Convertir y a float32
    y = tf.cast(y, tf.float32)
    
    # Generar un valor aleatorio para Mixup
    lambda_value = tf.random.uniform([], minval=0, maxval=1)
    
    batch_size = tf.shape(x)[0]
    index = tf.random.shuffle(tf.range(batch_size))

    # Aplicar Mixup a las imágenes y etiquetas
    mixed_x = lambda_value * x + (1 - lambda_value) * tf.gather(x, index)
    mixed_y = lambda_value * y + (1 - lambda_value) * tf.gather(y, index)
    return mixed_x, mixed_y


In [45]:

# Cargar el dataset desde directorios con preprocesamiento
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_DIR,
    labels="inferred",
    label_mode="categorical",  # Cambiar a 'categorical' para one-hot encoding
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    validation_split=0.2,
    subset="training",
    seed=SEED
)

validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_DIR,
    labels="inferred",
    label_mode="categorical",  # Cambiar a 'categorical' para one-hot encoding
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=SEED
)


# Aplicar Mixup al dataset de entrenamiento
train_ds = train_ds.map(lambda x, y: mixup((x, y)))

# Cachear y prefetch para optimización del pipeline
train_ds = train_ds.map(lambda x, y: mixup((x, y)), num_parallel_calls=tf.data.AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)


Found 7514 files belonging to 6 classes.
Using 6012 files for training.
Found 7514 files belonging to 6 classes.
Using 1502 files for validation.


In [46]:
import tensorflow as tf

# Obtener el número de clases manualmente, si se conoce
NUM_CLASSES = 6 # Reemplaza 5 con el número real de clases

# Cargar el modelo base preentrenado (MobileNetV2)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=INPUT_SHAPE,
    include_top=False,
    weights='imagenet'
)

# Congelar las capas iniciales del modelo
for layer in base_model.layers[:FINE_TUNE_POINT]:
    layer.trainable = False

# Construir el modelo completo
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255),  # Normalizar los valores de entrada
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(),   # Normalización de batch
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),       # Dropout para evitar overfitting
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compilar el modelo utilizando CategoricalCrossentropy y suavizado de etiquetas
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)


In [40]:

# Callbacks: EarlyStopping y ReduceLROnPlateau
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6
)

# Entrenar el modelo
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=EPOCHS,
    callbacks=[early_stopping, reduce_lr]
)

# Visualizar los resultados del entrenamiento
metrics = history.history
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['training', 'validation'])
plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.subplot(1, 2, 2)
plt.plot(history.epoch, metrics['accuracy'], metrics['val_accuracy'])
plt.legend(['training', 'validation'])
plt.ylabel('Accuracy')
plt.xlabel('Epoch')


Epoch 1/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 68s 587ms/step - accuracy: 0.3615 - loss: 4.3447 - val_accuracy: 0.3422 - val_loss: 3.4961 - learning_rate: 0.0010
Epoch 2/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 51s 539ms/step - accuracy: 0.4841 - loss: 2.9044 - val_accuracy: 0.4461 - val_loss: 2.7874 - learning_rate: 0.0010
Epoch 3/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 51s 537ms/step - accuracy: 0.5326 - loss: 2.2793 - val_accuracy: 0.3322 - val_loss: 2.2498 - learning_rate: 0.0010
Epoch 4/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 51s 540ms/step - accuracy: 0.5307 - loss: 1.9078 - val_accuracy: 0.4800 - val_loss: 1.8559 - learning_rate: 0.0010
Epoch 5/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 51s 536ms/step - accuracy: 0.5401 - loss: 1.6866 - val_accuracy: 0.4913 - val_loss: 1.6252 - learning_rate: 0.0010
Epoch 6/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 51s 541ms/step - accuracy: 0.5294 - loss: 1.5585 - val_accuracy: 0.4887 - val_loss: 1.5671 - learning_rate: 0.0010
Epoch 7/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 51s 537ms/step - accuracy: 0.5276 - loss: 1.

In [ ]:

# Evaluación del modelo y visualización de resultados adicionales para evitar el overfitting

# Evaluar el modelo en el conjunto de validación
val_results = model.evaluate(validation_ds, return_dict=True)
print("Resultados de evaluación en el conjunto de validación:")
for metric, value in val_results.items():
    print(f"{metric}: {value:.4f}")

# Predicción de etiquetas en el conjunto de validación
y_pred = model.predict(validation_ds)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.concatenate([y for _, y in validation_ds], axis=0)

# Reporte de clasificación
print("Reporte de clasificación en el conjunto de validación:")
print(classification_report(y_true, y_pred_classes, target_names=train_ds.class_names))

# Matriz de confusión
conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, xticklabels=train_ds.class_names, yticklabels=train_ds.class_names, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Matriz de Confusión')
plt.show()

# Curvas ROC y AUC para cada clase en el conjunto de validación
y_true_bin = label_binarize(y_true, classes=range(len(train_ds.class_names)))
plt.figure(figsize=(10, 8))

for i in range(len(train_ds.class_names)):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Clase {train_ds.class_names[i]} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curvas ROC para cada clase')
plt.legend(loc="lower right")
plt.show()
